In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import scvelo as scv
from scipy import stats
import os
import matplotlib
import pickle
%matplotlib inline
import palantir
import harmony

import random
np.random.seed(0)
random.seed(0)

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [2]:
mouse_counts = pd.read_csv('41586_2014_BFnature13173_MOESM31_ESM.txt', '\t', index_col = 0)
treut_anno = mouse_counts[['time_point','sample','putative_cell_type']]
treut_counts = mouse_counts.loc[:,
                                [i not in ['time_point','sample','putative_cell_type'] for i in mouse_counts.columns]]

treut_counts['Cluster'] = treut_anno.loc[treut_counts.index].putative_cell_type
treut_mean = treut_counts.groupby('Cluster').mean()
treut_mean

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007N19Rik,0610007P08Rik,0610007P14Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,0610009B22Rik,...,Zwilch,Zwint,Zxda,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3
Cluster,,,,,,,,,,,,,,,,,,,,,
AT1,0,1.834164,0.694008,2.817849,0.314491,1.472891,0.000000,0,0.000000,1.976590,...,0.000000,0.901919,0.202442,0.156888,0.152214,0,0.293534,0.965716,0.530506,0.733815
AT2,0,3.323112,1.153843,2.587494,0.199174,3.570385,0.000000,0,0.000000,1.751091,...,0.000000,0.944604,0.000000,0.016588,0.000000,0,0.273020,1.590353,0.508069,0.344981
BP,0,1.904798,1.828821,4.837435,0.444942,2.332611,0.268799,0,0.995508,3.164528,...,0.642436,0.906467,0.437528,0.470138,0.320796,0,0.271101,0.000000,0.203600,0.474671
Clara,0,3.976869,3.003600,1.042796,0.119054,3.269857,0.000000,0,0.788390,1.184740,...,0.000000,1.337765,0.582824,0.233838,0.538472,0,0.174552,3.480910,1.074533,0.720148
bulk,0,6.130456,2.886026,4.876307,0.249437,5.507640,4.243755,0,1.450930,5.411434,...,0.000000,2.792416,0.000000,0.000000,0.000000,0,0.000000,2.541019,0.553046,0.438479
ciliated,0,5.339313,0.000000,0.000000,1.379864,2.500572,0.000000,0,0.000000,5.746197,...,0.000000,0.979110,0.091880,0.000000,0.000000,0,0.813373,1.218893,1.734918,1.664028


In [3]:
mouse_counts = pd.read_csv('41586_2014_BFnature13173_MOESM31_ESM.txt', '\t', index_col = 0)
treut_anno = mouse_counts[['time_point','sample','putative_cell_type']]
treut_counts = mouse_counts.loc[:,
                                [i not in ['time_point','sample','putative_cell_type'] for i in mouse_counts.columns]]


In [4]:
#pd.Series(mouse_counts.columns.to_list()).to_csv('mouse_genes.tsv', sep = '\t', index = False)
h_m_mapper = pd.read_csv('../mouse_human_map.txt', sep = '\t', index_col = 1)['Gene name'].to_dict()
treut_counts.columns = treut_counts.columns.map(h_m_mapper)
treut_counts = treut_counts.loc[:,~treut_counts.columns.isna()]


In [6]:
treut_counts = treut_counts.T
treut_counts = treut_counts[~treut_counts.index.duplicated(keep='first')].T
treut_counts

,KIAA1841,C17orf49,NCBP2AS2,C2orf68,C4orf19,C11orf58,C9orf16,C9orf24,C11orf1,C3orf80,...,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
cell_name,,,,,,,,,,,,,,,,,,,,,
E18_2_C06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,2.504974,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.386314,0.000000
E18_2_C07,5.423807,0.000000,0.000000,3.202661,0.000000,4.836374,0.000000,0.000000,7.566442,0.0,...,0.000000,0.000000,8.351668,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
E18_2_C11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.276239,0.000000,2.818468,0.000000
E18_2_C12,0.000000,0.000000,0.000000,3.514148,0.000000,4.037611,0.000000,0.000000,0.000000,0.0,...,0.000000,2.601014,0.000000,0.000000,0.0,0.0,4.533582,0.000000,0.000000,0.225300
E18_2_C13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.278452,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E18_1_C62,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.700190,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
E18_1_C67,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.380115,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,6.786832,0.0,0.0,0.000000,6.043891,0.000000,0.000000
E18_1_C84,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.638559,0.0,...,0.000000,1.896876,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [11]:
treut_counts.T.to_csv('Treutlein_counts.tsv', sep = '\t')

In [10]:
treut_anno['Cell-type'] = treut_anno['putative_cell_type']
treut_anno['Dataset'] = 'Treutlein_mouse'
treut_anno.to_csv('Treutlein_meta.tsv', sep = '\t')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
